In [2]:
import pandas as pd
import os

In [28]:
def get_file_list(base_path='/neurospin/dico/data/human/dHCP/derivatives/release1/morphometry/morpho_talairach'):
    files= []
    # Get the list of files in the directory
    for file in os.listdir(base_path):
        # Check if the file ends with .csv
        if file.endswith(".dat"):
            files.append(file)

    return files

def get_sulci_list(base_path='/neurospin/dico/data/human/dHCP/derivatives/release1/morphometry/morpho_talairach'):
    files = get_file_list(base_path)
    sulci_list = [x.split('.dat')[0].split('morpho_')[1] for x in files]

    return sulci_list

In [29]:
print(get_sulci_list())

['F.Cal.ant.-Sc.Cal._left', 'F.I.P.r.int.1_left', 'S.Pe.C.marginal._left', 'F.C.L.r.retroC.tr._left', 'F.C.L.r.sc.post._left', 'S.T.i.post._left', 'F.C.L.r.asc._left', 'S.F.sup._left', 'S.F.marginal._left', 'F.P.O._left', 'S.Call._left', 'S.R.sup._left', 'ventricle_left', 'S.Pe.C.inter._left', 'S.O.T.lat.ant._left', 'S.C._left', 'F.C.L.a._left', 'S.C.LPC._left', 'S.Olf._left', 'S.Po.C.sup._left', 'S.Pe.C.median._left', 'S.Li._left', 'S.Or._left', 'S.F.median._left', 'S.F.inf._left', 'F.C.L.r.diag._left', 'S.T.s.ter.asc.ant._left', 'S.Pa.sup._left', 'S.Rh._left', 'S.F.orbitaire._left', 'S.T.s.ter.asc.post._left', 'S.R.inf._left', 'S.Pe.C.inf._left', 'OCCIPITAL_left', 'F.C.M.r.AMS.ant._left', 'F.C.L.r.sc.ant._left', 'S.Cu._left', 'F.C.M.post._left', 'F.C.M.ant._left', 'S.T.pol._left', 'S.O.p._left', 'F.C.L.p._left', 'S.F.int._left', 'S.Pa.t._left', 'S.Or.l._left', 'INSULA_left', 'S.F.inter._left', 'S.F.polaire.tr._left', 'S.Pa.int._left', 'S.O.T.lat.post._left', 'S.p.C._left', 'F.I.P._le

In [71]:
def get_gravity_center_coordinates(base_path , sulci):

    
    # Read the sulci tsv
    file_path = os.path.join(base_path, sulci)

    sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]

    # Get the coordinates of the gravity center

    return sulci_df

In [72]:
sulci = get_file_list()[0]

df_test = get_gravity_center_coordinates(base_path="/neurospin/dico/data/human/dHCP/derivatives/release1/morphometry/morpho_talairach", sulci=sulci)

print(df_test.head())


   gravityCenter_x  gravityCenter_y  gravityCenter_z
0          13.1009          73.3183        -5.135020
1          16.1079          72.3311        -1.926680
2          13.1828          74.0018         0.871518
3          16.7162          76.4786       -12.863700
4          12.3674          79.9126        -6.443830


/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]


In [73]:
def compute_gravity_center_coordinates_shift(sulci):

    """
    Compute the shift of the gravity center coordinates between babies and adults for a given sulcus
    """

    # Get the coordinates of the gravity center for babies
    base_path_babies = "/neurospin/dico/data/human/dHCP/derivatives/release1/morphometry/morpho_talairach"
    df_babies = get_gravity_center_coordinates(base_path_babies, sulci)

    # Get the coordinates of the gravity center for adults
    base_path_adults = "/neurospin/dico/data/human/pclean/derivatives/morphologist-2021/morphometry/3.3_base2018/morpho_talairach"
    df_adults = get_gravity_center_coordinates(base_path_adults, sulci)

    # Create a dataframe with the coordinates shift and the distance between the two gravity centers

    mean_babies = df_babies.mean()
    mean_adults = df_adults.mean()

    shift_x = mean_adults["gravityCenter_x"] - mean_babies["gravityCenter_x"]
    shift_y = mean_adults["gravityCenter_y"] - mean_babies["gravityCenter_y"]
    shift_z = mean_adults["gravityCenter_z"] - mean_babies["gravityCenter_z"]
    distance = (shift_x**2 + shift_y**2 + shift_z**2)**0.5

    sulcus_name = sulci.split('.dat')[0].split('morpho_')[1]

    df_shift_sulcus= pd.DataFrame([{
        "sulcus": sulcus_name,
        "shift_x": shift_x,
        "shift_y": shift_y,
        "shift_z": shift_z,
        "distance": distance
    }])

    return df_shift_sulcus
     


In [74]:
# Test the function
sulci = get_file_list()[0]
df_test = compute_gravity_center_coordinates_shift(sulci)
print(df_test.head())

                    sulcus   shift_x   shift_y   shift_z  distance  n_babies  \
0  F.Cal.ant.-Sc.Cal._left -3.001255 -3.049756  2.191675  4.807492        40   

   n_adults  
0        62  


/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]
/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]


In [75]:
def compute_all_gravity_center_shifts():

    # Base paths
    base_path_babies = "/neurospin/dico/data/human/dHCP/derivatives/release1/morphometry/morpho_talairach"
    base_path_adults = "/neurospin/dico/data/human/pclean/derivatives/morphologist-2021/morphometry/3.3_base2018/morpho_talairach"

    # Get all the .dat files
    sulci_files = get_file_list(base_path_babies)

    # Create an empty list
    all_shifts = []

    for sulcus_file in sulci_files:
        df_shift = compute_gravity_center_coordinates_shift(sulcus_file)
        if df_shift is not None:
            all_shifts.append(df_shift)

    df_all_shifts = pd.concat(all_shifts, ignore_index=True)

    # Sauvegarde en CSV
    df_all_shifts.to_csv("sulci_gravity_center_shifts.csv", index=False)

    # Stats : sulci les plus décalés en moyenne
    df_summary = df_all_shifts.groupby("sulcus")["distance"].mean().sort_values(ascending=False)
    print(df_summary.head(10))



In [76]:
compute_all_gravity_center_shifts()

/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]
/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]
/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]
/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyw

sulcus
S.R.sup._left            49.085198
S.O.p._left              27.140924
ventricle_left           26.007206
S.Pe.C.marginal._left    20.644260
S.F.int._left            14.644574
F.C.L.r.sc.ant._left     13.449479
S.Pe.C.sup._left         12.245116
F.C.L.r.sc.post._left    12.213125
S.T.s._left              11.684260
S.F.polaire.tr._left     11.617511
Name: distance, dtype: float64


/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]
/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]
/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  sulci_df = pd.read_csv(file_path, delim_whitespace=True).loc[:,["gravityCenter_x", "gravityCenter_y", "gravityCenter_z"]]
/tmp/ipykernel_2130437/1177162784.py:7: FutureWarning: The 'delim_whitespace' keyw